# The following are the parameter spaces used for evaluations

## The parameter search space used by Tuner for xgboost, KNN and SVM in the Section IV-A2.

In [1]:
from scipy.stats import uniform
from mango.domain.distribution import loguniform

param_svm = {'gamma':uniform(0.1, 4),
             'C':loguniform(-7, 10)}

param_xgboost = {'learning_rate':uniform(0, 1),
                 'gamma':uniform(0, 5),
                 'max_depth':range(1,11),
                 'n_estimators':range(1,301),
                 'booster':['gbtree','gblinear','dart']}
               
param_knn = {'n_neighbors':range(1, 51),
             'algorithm':['auto','ball_tree','kd_tree',
             'brute']}

## The parameter search space used by MetaTuner for neural network, decision tree, xgboost, KNN and SVM in the Section IV-A3. The neural network has one hidden layer with number of hidden nodes specified in the search space. 

In [2]:
from scipy.stats import uniform
from mango.domain.distribution import loguniform

param_nn = {'type':'clf_nn',
            'num_of_nodes':range(10, 101)}

param_dtree = {'type':'clf_dtree',
               'max_features':['auto', 'sqrt', 'log2'],
               'max_depth':range(1,21),     
               'splitter':['best','random'],
               'criterion':['gini','entropy']}
             
param_svm = {'type':'clf_svm',
             'gamma':uniform(0.1, 4),
             'C':loguniform(-7, 10)}

param_xgboost = {'type':'clf_xgboost',
                 'learning_rate':uniform(0, 1),
                 'gamma':uniform(0, 5),
                 'max_depth':range(1,21),
                 'n_estimators':range(1,11),
                 'booster':['gbtree','gblinear','dart']}
               
param_knn = {'type':'clf_knn',
             'n_neighbors': range(1, 51),
             'algorithm':['auto','ball_tree','kd_tree', 
             'brute']}
